# Premature babies | gestational age & Cooling therapy treatment

In [2]:
headers = {
  'Accept': 'application/fhir+json; fhirVersion=4.0',
  'Content-Type': 'application/fhir+json; fhirVersion=4.0',
  'Cookie': 'ARRAffinity=a6e48b9e9d2653435be7b61998d8624b44115214104213d6c8b8c526cc56dc70; ARRAffinitySameSite=a6e48b9e9d2653435be7b61998d8624b44115214104213d6c8b8c526cc56dc70'
}

In [3]:
import uuid
import pandas as pd
import requests
import json
import datetime
import random
import pytz
import pandas as pd


In [4]:
df1 = pd.DataFrame(columns=['year', 'gestational age'])
df2 = pd.DataFrame(columns=['year'])

def generate_random_datetime(year):
    start_date = datetime.datetime(year, 1, 1)
    end_date = datetime.datetime(year, 12, 31)

    # Calculate the total number of seconds within the specified year
    total_seconds = (end_date - start_date).total_seconds()

    # Generate a random duration within the specified year
    random_duration = random.randint(0, int(total_seconds))

    # Add the random duration to the start date
    random_datetime = start_date + datetime.timedelta(seconds=random_duration)

    # Convert the random datetime to the desired format
    formatted_datetime = random_datetime.strftime("%Y-%m-%dT%H:%M:%S")
    formatted_datetime = str(formatted_datetime)+'+02:00'

    return formatted_datetime

def generate_random_gestational_age():
    # Generate a random gestational age with less prematurity during COVID times
    if random.random() < 0.7:  # 70% chance of being full term
        # source: https://pathways.org/adjusting-age-for-prematurity/
        gestational_age = random.randint(38, 42)  # Full-term range (38-42 weeks)
    else:
        gestational_age = random.randint(28, 37)  # Preterm range (28-37 weeks)

    return gestational_age

def generate_dfs(datetime, gestational_age):
    # Placeholder function to generate an observation
    # Replace this function with your actual implementation
    # You can access the datetime and gestational_age variables here
    # to generate the observation data for each patient
    #print(f"Observation generated for datetime: {datetime} and gestational age: {gestational_age}")
    
    df1.loc[len(df1)] = {'year': str(datetime)[:4], 'gestational age': gestational_age}


In [5]:
def generate_patients():
    years = range(1960,2023)
    for num_pat in range(0,20):
        birth = generate_random_datetime(years[num_pat])
        data = {
            "resourceType": "Patient",
            "id": f"patient-{str(num_pat)}",
            "active": True,
            "name": [
                {
                    "use": "official",
                    "family": "Patient dumb",
                    "given": [
                        "Dumb","Patient"
                    ]
                },
                {
                    "use": "usual",
                    "given": [
                        "Dumb pat"
                    ]
                }
            ],
            "gender": "female",
            "birthDate": str(birth).split('T')[0]
        }
        url = f"https://firelyasserver.azurewebsites.net/Patient/patient-{str(num_pat)}"
        payload = json.dumps(data)
        response = requests.request("PUT", url, data=payload, headers=headers)
        if response.status_code != 201 and response.status_code != 200:
            print(f'Female patient not created {num_pat}')
            print(payload)

    for num_pat_male in range(0,20):
        birth = generate_random_datetime(years[num_pat_male])
        data = {
            "resourceType": "Patient",
            "id": f"patient-male-{str(num_pat_male)}",
            "active": True,
            "name": [
                {
                    "use": "official",
                    "family": "Patient dumb",
                    "given": [
                        "Dumb","Patient"
                    ]
                },
                {
                    "use": "usual",
                    "given": [
                        "Dumb pat"
                    ]
                }
            ],
            "gender": "male",
            "birthDate": str(birth).split('T')[0]
        }
        url = f"https://firelyasserver.azurewebsites.net/Patient/patient-male-{str(num_pat_male)}"
        payload = json.dumps(data)
        response = requests.request("PUT", url, data=payload, headers=headers)
        if response.status_code != 201 and response.status_code != 200:
            print(f'Male patient not created {num_pat_male}')

In [6]:
def generate_observation(datetime, value):
    ID = str(uuid.uuid4())
    random_patient = random.randint(0, 20)
    observation = {
        "resourceType": "Observation",
        "id": ID,
        "status": "final",
        "category": [
            {
                "coding": [
                    {
                        "system": "gestational-age",
                        "code": "gestational-age",
                        "display": "Gestational age"
                    }
                ]
            }
        ],
        "code": {
            "coding": [
                {
                    "system": "gestAge",
                    "code": "gestAge",
                    "display": "Gestational Age"
                }
            ],
            "text": "Gestational Age"
        },
        "subject": {
            "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}"
        },
        "encounter": {
            "reference": "no-ref"
        },
        "effectiveDateTime": datetime #"2020-08-31T22:13:22+02:00",
        ,
        "issued": "2020-08-31T22:13:22.922+02:00",
        "valueQuantity": {
            "value": value,
            "unit": "weeks"
        },
        "interpretation" : [{
            "coding" : [{
            "system" : "http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation",
            "code" : "H",
            "display" : "High"
            }]
        }],
        "bodySite" : {
            "coding" : [{
            "system" : "http://snomed.info/sct",
            "code" : f"{random_patient}",
            "display" : f"Body part {random_patient}"
            }]
        },
        "component" : [{
            "code" : {
            "coding" : [{
                "system" : "http://loinc.org",
                "code" : f"{random_patient}",
                "display" : f"Component {random_patient}"
            }]
            }
        }],
    }
    url = f"https://firelyasserver.azurewebsites.net/Observation/{ID}"
    print(observation)
    payload = json.dumps(observation)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Observation ot created')
        print(payload)


In [7]:
def generate_procedure_coolingtherapy(datetime):
    ID = str(uuid.uuid4())
    random_patient = random.randint(0, 20)
    procedure = {
                "resourceType": "Procedure",
                "id": ID,
                "status": "completed",
                "code": {
                    "coding": [
                        {
                            "system": "CT",
                            "code": "CT",
                            "display": "Cooling therapy"
                        }
                    ],
                    "text": "Cooling therapy"
                },
                "subject": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}"
                },
                "encounter": {
                    "reference": "no-ref"
                },
                "performedDateTime": datetime,
                "location": {
                    "reference": "AMC",
                    "display": "AMC"
                },
                "category" : [{
                    "coding" : [{
                    "system" : "http://snomed.info/sct",
                    "code" : f"{random_patient}",
                    "display" : f"category {random_patient}"
                    }],
                    "text" : "Education"
                }],
                "outcome" : {
                    "text" : "success"
                },
                "note" : [{
                    "text" : f"Note {random_patient}"
                }],
            }
    url = f"https://firelyasserver.azurewebsites.net/Procedure/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Procedure not created')

In [8]:
def generate_medication():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    procedure = {
                "resourceType": "Medication",
                "meta": {
                    "tag": [
                        {
                            "system": "http://www.alpha.alp/use-case",
                            "code": "MLOZ"
                        }
                    ],
                    "versionId": "46aba869-47f6-4171-8c01-eae5cc9f00c3",
                    "lastUpdated": "2023-05-31T06:41:53.310+00:00"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><b>Medication:</b> Erythromycin 250 MG Oral Tablet\n          </div>"
                },
                "code": {
                    "coding": [
                        {
                            "system": "http://www.nlm.nih.gov/research/umls/rxnorm",
                            "code": f"{random_code}",
                            "display": f"Medicine {random_code}"
                        }
                    ],
                    "text": f"Medicine {random_code}"
                },
                "id": f"{ID}",
                "status" : "active"
    }
    url = f"https://firelyasserver.azurewebsites.net/Medication/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('medicine not created')
        print(payload)


In [9]:
def generate_adverseevent():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2024)
    procedure = {
                "resourceType": "AdverseEvent",
                "id": f"{ID}",
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Generated Narrative with Details</b></p><p><b>id</b>: example</p><p><b>identifier</b>: 49476534</p><p><b>actuality</b>: actual</p><p><b>category</b>: Product Use Error <span>(Details : {http://terminology.hl7.org/CodeSystem/adverse-event-category code 'product-use-error' = 'Product Use Error', given as 'Product Use Error'})</span></p><p><b>event</b>: This was a mild rash on the left forearm <span>(Details : {SNOMED CT code '304386008' = 'O/E - itchy rash', given as 'O/E - itchy rash'})</span></p><p><b>subject</b>: <a>Patient/example</a></p><p><b>date</b>: 29/01/2017 12:34:56 PM</p><p><b>seriousness</b>: Non-serious <span>(Details : {http://terminology.hl7.org/CodeSystem/adverse-event-seriousness code 'Non-serious' = 'Non-serious', given as 'Non-serious'})</span></p><p><b>severity</b>: Mild <span>(Details : {http://terminology.hl7.org/CodeSystem/adverse-event-severity code 'mild' = 'Mild', given as 'Mild'})</span></p><p><b>recorder</b>: <a>Practitioner/example</a></p><h3>SuspectEntities</h3><table><tr><td>-</td><td><b>Instance</b></td></tr><tr><td>*</td><td><a>Medication/example</a></td></tr></table></div>"
                },
                "identifier": {
                    "system": "http://acme.com/ids/patients/risks",
                    "value": "49476534"
                },
                "actuality": "actual",
                "category": [
                    {
                        "coding": [
                            {
                                "system": "http://terminology.hl7.org/CodeSystem/adverse-event-category",
                                "code": f"{random_code}",
                                "display": f"Adverse event {random_code}"
                            }
                        ]
                    }
                ],
                "event": {
                    "coding": [
                        {
                            "system": "http://snomed.info/sct",
                            "code": f"{random_code}",
                            "display": f"Adverse Event {random_code}"
                        }
                    ],
                    "text": f"Adverse Event {random_code}"
                },
                "subject": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}"
                },
                "date": f"{random_year}-01-29T12:34:56+00:00",
                "seriousness": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/adverse-event-seriousness",
                            "code": "serious",
                            "display": f"Seriousness {random_code}"
                        }
                    ]
                },
                "severity": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/adverse-event-severity",
                            "code": "moderate",
                            "display": f"Severity {random_code}"
                        }
                    ]
                },
                "recorder": {
                    "reference": "https://server.fire.ly/Practitioner/example"
                },
                "suspectEntity": [
                    {
                        "instance": {
                            "reference": "https://server.fire.ly/Medication/example"
                        }
                    }
                ],
                "meta": {
                    "versionId": "6729c795-0caa-4ab5-8984-f4c0afeccdcf",
                    "lastUpdated": "2023-03-21T07:08:58.673+00:00"
                }
            }
    url = f"https://firelyasserver.azurewebsites.net/AdverseEvent/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Adverse Event not created')
        print(response.status_code)
        print(payload)


In [10]:
def generate_encounter():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "Encounter",
                "id": f"{ID}",
                "meta": {
                    "profile": [
                        "http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter"
                    ],
                    "versionId": "5dfa233a-c5e1-49d9-9563-893075ea43cd",
                    "lastUpdated": "2023-06-03T17:18:51.082+00:00"
                },
                "identifier": [
                    {
                        "use": "official",
                        "system": "https://github.com/synthetichealth/synthea",
                        "value": "3d4fdbda-7661-4068-9041-43ab8046bf15"
                    }
                ],
                "status": "finished",
                "class": {
                    "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
                    "code": f"{random_code}"
                },
                "type": [
                    {
                        "coding": [
                            {
                                "system": "http://snomed.info/sct",
                                "code":f"{random_code}",
                                "display": f"Encounter {random_code}"
                            }
                        ],
                        "text": "Encounter for symptom"
                    }
                ],
                "subject": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}",
                },
                "participant": [
                    {
                        "type": [
                            {
                                "coding": [
                                    {
                                        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
                                        "code": "PPRF",
                                        "display": "primary performer"
                                    }
                                ],
                                "text": "primary performer"
                            }
                        ],
                        "period": {
                            "start": f"{random_year}-10-28T17:26:18-04:00",
                            "end": f"{random_year}-10-28T17:55:18-04:00"
                        },
                        "individual": {
                            "reference": "https://server.fire.ly/Practitioner/test-uscore4",
                            "display": "Dr. Adriana394 Alvarez441"
                        }
                    }
                ],
                "period": {
                    "start": f"{random_year}-10-28T17:26:18-04:00",
                    "end": f"{random_year}-10-28T17:55:18-04:00"
                },
                "reasonCode": [
                    {
                        "coding": [
                            {
                                "system": "http://snomed.info/sct",
                                "code":f"{random_code}",
                                "display": f"Encounter {random_code}"
                            }
                        ]
                    }
                ],
                "hospitalization": {
                    "dischargeDisposition": {
                        "coding": [
                            {
                                "code": "home",
                                "system": "http://terminology.hl7.org/CodeSystem/discharge-disposition"
                            }
                        ]
                    }
                },
                "location": [
                    {
                        "location": {
                            "reference": "https://server.fire.ly/Location/test-uscore4",
                            "display": "BERKSHIRE MEDICAL CENTER INC - 1"
                        }
                    }
                ],
                "serviceProvider": {
                    "reference": "https://server.fire.ly/Organization/test-uscore4",
                    "display": "BERKSHIRE MEDICAL CENTER INC - 1"
                },
                "priority" : {
                    "coding" : [{
                    "system" : "http://snomed.info/sct",
                    "code" : f"{random_code}",
                    "display" : f"Priority {random_code}"
                    }]
                },
                "length" : {
                    "value" : random_code,
                    "unit" : "min",
                    "system" : "http://unitsofmeasure.org",
                    "code" : "min"
                },
                "location" : [{
                    "location" : {
                        "reference" : "home",
                        "display" : "Client's home"
                    },
                    "status" : "completed",
                    "period" : {
                        "start" : "2015-01-17T16:00:00+10:00",
                        "end" : "2015-01-17T16:30:00+10:00"
                    }
                }]

            }
    url = f"https://firelyasserver.azurewebsites.net/Encounter/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Encounter not created')
        print(response.status_code)
        print(payload)


In [11]:
def generate_allergy():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    procedure = {
                "resourceType": "AllergyIntolerance",
                "id": f"{ID}",
                "meta": {
                    "profile": [
                        "http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance"
                    ],
                    "versionId": "51eecb1c-7116-4825-bb17-0cfccc10a3bd",
                    "lastUpdated": "2023-06-03T17:18:50.938+00:00"
                },
                "clinicalStatus": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
                            "code": "active"
                        }
                    ]
                },
                "verificationStatus": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
                            "code": "confirmed"
                        }
                    ]
                },
                "type": "allergy",
                "category": [
                    "food"
                ],
                "criticality": "low",
                "code": {
                    "coding": [
                        {
                            "system": "http://snomed.info/sct",
                            "code": f"{random_code}",
                            "display": f"Allergy to {random_code}"
                        }
                    ],
                    "text": f"Allergy to {random_code}"
                },
                "patient": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}",
                },
                "recordedDate": "1970-03-17T16:26:18-05:00",
                "reaction": [
                    {
                        "manifestation": [
                            {
                                "coding": [
                                    {
                                        "system": "http://snomed.info/sct",
                                        "code": f"{random_code}",
                                        "display": f"Recation {random_code}"
                                    }
                                ]
                            }
                        ]
                    }
                ]
            }
    url = f"https://firelyasserver.azurewebsites.net/AllergyIntolerance/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('AllergyIntolerance not created')
        print(payload)


In [12]:
def generate_careplan():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "CarePlan",
                "id": f"{ID}",
                "meta": {
                    "profile": [
                        "http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan"
                    ],
                    "versionId": "f1e39ccf-5cbf-4aa8-8b54-71bf31675b3c",
                    "lastUpdated": "2023-06-03T17:18:50.952+00:00"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Care Plan for Self-care interventions (procedure).<br/>Activities: <ul><li>Self-care interventions (procedure)</li><li>Self-care interventions (procedure)</li><li>Self-care interventions (procedure)</li></ul></div>"
                },
                "status": "active",
                "intent": "order",
                "category": [
                    {
                        "coding": [
                            {
                                "system": "http://hl7.org/fhir/us/core/CodeSystem/careplan-category",
                                "code": f"{random_code}",
                                "display": f"Category {random_code}"
                            }
                        ]
                    },
                    {
                        "coding": [
                            {
                                "system": "http://snomed.info/sct",
                                "code": f"{random_code}",
                                "display": f"Self-care {random_code} (procedure)"
                            }
                        ],
                        "text": f"Self-care {random_code} (procedure)"
                    }
                ],
                "subject": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}",
                },
                "encounter": {
                    "reference": "https://server.fire.ly/Encounter/test-uscore4"
                },
                "period": {
                    "start": f"{random_year}-03-01T16:26:18-05:00"
                },
                "careTeam": [
                    {
                        "reference": "https://server.fire.ly/CareTeam/test-uscore4"
                    }
                ]
            }
    url = f"https://firelyasserver.azurewebsites.net/CarePlan/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Careplan not created')
        print(payload)


In [13]:
def generate_claim():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "ClaimResponse",
                "id": f"{ID}",
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">A human-readable rendering of the ClaimResponse</div>"
                },
                "identifier": [
                    {
                        "system": "http://www.BenefitsInc.com/fhir/remittance",
                        "value": f"{random_code}"
                    }
                ],
                "status": "active",
                "type": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/claim-type",
                            "code": "oral"
                        }
                    ]
                },
                "subType": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/ex-claimsubtype",
                            "code": "emergency"
                        }
                    ]
                },
                "use": "claim",
                "patient": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}",
                },
                "created": f"{random_year}-08-16",
                "insurer": {
                    "identifier": {
                        "system": "http://www.jurisdiction.org/insurers",
                        "value": "555123"
                    }
                },
                "requestor": {
                    "reference": "https://server.fire.ly/Organization/1"
                },
                "request": {
                    "reference": "http://www.BenefitsInc.com/fhir/oralhealthclaim/15476332402"
                },
                "outcome": "complete",
                "disposition": "Claim settled as per contract.",
                "payeeType": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/payeetype",
                            "code": "provider"
                        }
                    ]
                },
                "item": [
                    {
                        "itemSequence": 1,
                        "adjudication": [
                            {
                                "category": {
                                    "coding": [
                                        {
                                            "code": "eligible"
                                        }
                                    ]
                                },
                                "amount": {
                                    "value": random_code,
                                    "currency": "USD"
                                }
                            },
                            {
                                "category": {
                                    "coding": [
                                        {
                                            "code": "copay"
                                        }
                                    ]
                                },
                                "amount": {
                                    "value": random_code,
                                    "currency": "USD"
                                }
                            },
                            {
                                "category": {
                                    "coding": [
                                        {
                                            "code": "eligpercent"
                                        }
                                    ]
                                },
                                "value":random_code
                            },
                            {
                                "category": {
                                    "coding": [
                                        {
                                            "code": "benefit"
                                        }
                                    ]
                                },
                                "reason": {
                                    "coding": [
                                        {
                                            "system": "http://terminology.hl7.org/CodeSystem/adjudication-reason",
                                            "code": "ar002",
                                            "display": "Plan Limit Reached"
                                        }
                                    ]
                                },
                                "amount": {
                                    "value": random_code,
                                    "currency": "USD"
                                }
                            }
                        ]
                    }
                ],
                "total": [
                    {
                        "category": {
                            "coding": [
                                {
                                    "code": "submitted"
                                }
                            ]
                        },
                        "amount": {
                            "value": random_code,
                            "currency": "USD"
                        }
                    },
                    {
                        "category": {
                            "coding": [
                                {
                                    "code": "benefit"
                                }
                            ]
                        },
                        "amount": {
                            "value": random_code,
                            "currency": "USD"
                        }
                    }
                ],
                "payment": {
                    "type": {
                        "coding": [
                            {
                                "system": "http://terminology.hl7.org/CodeSystem/ex-paymenttype",
                                "code": "complete"
                            }
                        ]
                    },
                    "date": f"{random_year}-08-31",
                    "amount": {
                        "value": random_code,
                        "currency": "USD"
                    },
                    "identifier": {
                        "system": "http://www.BenefitsInc.com/fhir/paymentidentifier",
                        "value": "201408-2-1569478"
                    }
                },
                "meta": {
                    "versionId": "c9c4276e-39ba-4b59-9144-cc16fbe9d595",
                    "lastUpdated": "2023-01-26T11:36:53.5073032+00:00"
                }
            }
    url = f"https://firelyasserver.azurewebsites.net/ClaimResponse/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Claimresponse not created')

In [14]:
def generate_condition():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "Condition",
                "id": f"{ID}",
                "meta": {
                    "profile": [
                        "http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition"
                    ],
                    "versionId": "3f48f9f6-b9d8-4815-99cc-cc813a91975c",
                    "lastUpdated": "2023-06-03T17:18:51.007+00:00"
                },
                "clinicalStatus": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
                            "code": "resolved"
                        }
                    ]
                },
                "verificationStatus": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/condition-ver-status",
                            "code": "confirmed"
                        }
                    ]
                },
                "category": [
                    {
                        "coding": [
                            {
                                "system": "http://terminology.hl7.org/CodeSystem/condition-category",
                                "code": f"{random_code}",
                                "display": f"Condition {random_code}"
                            }
                        ]
                    }
                ],
                "severity" : {
                    "coding" : [{
                    "system" : "http://snomed.info/sct",
                    "code" : f"{random_code}",
                    "display" : f"Severity {random_code}"
                    }]
                },
                "code": {
                    "coding": [
                        {
                            "system": "http://snomed.info/sct",
                            "code": f"{random_code}",
                            "display": f"Condition {random_code}"
                        }
                    ],
                    "text": f"Condition {random_code}"
                },
                "subject": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}",
                },
                "encounter": {
                    "reference": "https://server.fire.ly/Encounter/test-uscore4"
                },
                "onsetDateTime": f"{random_year}-04-24T17:26:18-04:00",
                "abatementDateTime": f"{random_year}-05-08T17:26:18-04:00",
                "recordedDate": f"{random_year}-04-24T17:26:18-04:00"
            }
    url = f"https://firelyasserver.azurewebsites.net/Condition/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Condition not created')

In [15]:
def generate_detectedissue():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "DetectedIssue",
                "id": f"{ID}",
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">\n      <p><b>Severity: High</b> - Risk of internal bleeding</p>\n      <p>Those who take acetaminophen along with the widely used blood-thinning drug warfarin may face the risk of serious internal bleeding. People on warfarin who take acetaminophen for at least seven days in a row should be closely watched for bleeding.</p>\n      <ul>\n        <li><a href=\"MedicationStatement/tylenol\">500 mg Acetaminophen tablet 1/day, PRN since 2010</a></li>\n        <li><a href=\"MedicationRequest/warfarin\">Warfarin 1 MG TAB prescribed Jan. 5, 2014</a></li>\n      </ul>\n      <p>Mitigation: Jan 5, 2014 by Dr. Adam Careful: </p>\n      <p>Asked patient to discontinue regular use of Tylenol and to consult with clinician if they need to resume to allow appropriate INR monitoring</p>\n    </div>"
                },
                "status": "final",
                "code": {
                    "coding": [
                        {
                            "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
                            "code": f"{random_code}",
                            "display": f"Drug {random_code}"
                        }
                    ]
                },
                "severity": "high",
                "identifiedDateTime": f"{random_year}-01-05",
                "author": {
                    "reference": "https://server.fire.ly/Device/software"
                },
                "implicated": [
                    {
                        "reference": "https://server.fire.ly/MedicationStatement/example001",
                        "display": "500 mg Acetaminophen tablet 1/day, PRN since 2010"
                    },
                    {
                        "reference": "https://server.fire.ly/MedicationRequest/medrx0331",
                        "display": "Warfarin 1 MG TAB prescribed Jan. 15, 2015"
                    }
                ],
                "detail": f"Detail {random_code}",
                "mitigation": [
                    {
                        "action": {
                            "coding": [
                                {
                                    "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
                                    "code": f"{random_code}",
                                    "display": "Stopped Concurrent Therapy"
                                }
                            ],
                            "text": "Asked patient to discontinue regular use of Tylenol and to consult with clinician if they need to resume to allow appropriate INR monitoring"
                        },
                        "date": f"{random_year}-01-05",
                        "author": {
                            "reference": "https://server.fire.ly/Practitioner/example",
                            "display": "Dr. Adam Careful"
                        }
                    }
                ],
                "meta": {
                    "versionId": "cee81128-dc81-4538-8486-da8152554b07",
                    "lastUpdated": "2023-01-26T11:36:54.6750366+00:00"
                }
            }
    url = f"https://firelyasserver.azurewebsites.net/DetectedIssue/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Detected issue not created')

In [16]:
def generate_insurance():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "InsurancePlan",
                "id": f"{ID}",
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Generated Narrative with Details</b></p><p><b>id</b>: example</p><p><b>name</b>: foo</p></div>"
                },
                "name": f"{random_code}",
                "meta": {
                    "versionId": "6bf4e14f-b680-4514-b5c8-e6eb42e2fd22",
                    "lastUpdated": "2023-01-26T11:36:54.7064614+00:00"
                }
            }
    url = f"https://firelyasserver.azurewebsites.net/InsurancePlan/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Insurance not created')

In [17]:
def generate_RiskAssesment():
    ID = str(uuid.uuid4())
    random_code = random.randint(0, 10)
    random_patient = random.randint(0, 20)
    random_year = random.randint(2017,2023)
    procedure = {
                "resourceType": "RiskAssessment",
                "id": f"{ID}",
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Risk assessment for reference only</div>"
                },
                "identifier": [
                    {
                        "use": "official",
                        "system": "http://example.org",
                        "value": "risk-assessment-breastcancer1"
                    }
                ],
                "basedOn": {
                    "reference": "https://server.fire.ly/ImmunizationRecommendation/example"
                },
                "parent": {
                    "reference": "https://server.fire.ly/DiagnosticReport/example"
                },
                "status": "final",
                "code": {
                    "coding": [
                        {
                            "system": "http://browser.ihtsdotools.org/",
                            "code": f"{random_year}",
                            "display": "Assessment of risk for disease (procedure)"
                        }
                    ]
                },
                "subject": {
                    "reference": f"https://firelyasserver.azurewebsites.net/Patient/patient-{random_patient}",
                },
                "encounter": {
                    "reference": "https://server.fire.ly/Encounter/example",
                    "display": "Encounter with patient @example"
                },
                "occurrenceDateTime": f"{random_year}-10-10",
                "condition": {
                    "reference": "https://server.fire.ly/Condition/example"
                },
                "performer": {
                    "reference": "https://server.fire.ly/Practitioner/example"
                },
                "basis": [
                    {
                        "reference": "https://server.fire.ly/DiagnosticReport/example"
                    },
                    {
                        "reference": "https://server.fire.ly/Observation/example"
                    }
                ],
                "prediction": [
                    {
                        "outcome": {
                            "text": "Breast Cancer"
                        },
                        "probabilityDecimal": 0.000368+random_code,
                        "whenRange": {
                            "low": {
                                "value": 54,
                                "unit": "years",
                                "system": "http://unitsofmeasure.org",
                                "code": "a"
                            },
                            "high": {
                                "value": 57,
                                "unit": "years",
                                "system": "http://unitsofmeasure.org",
                                "code": "a"
                            }
                        }
                    }
                ],
                "note": [
                    {
                        "text": "This risk assessment is for reference only"
                    }
                ],
                "meta": {
                    "versionId": "bd7f0b86-b5e8-433d-a7ab-17dd1f657258",
                    "lastUpdated": "2023-01-26T11:36:55.3750825+00:00"
                }
            }
    url = f"https://firelyasserver.azurewebsites.net/RiskAssessment/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Risk not created')


In [18]:
generate_patients()


In [66]:
for i in range(0,100):
    generate_adverseevent()
    generate_allergy()
    generate_careplan()
    generate_claim()
    generate_condition()
    generate_detectedissue()
    generate_encounter()
    generate_insurance()
    generate_medication()
    generate_RiskAssesment()

In [74]:

for year in range(2017, 2024):
    for _ in range(100):
        random_datetime = generate_random_datetime(year)
        random_gestational_age = generate_random_gestational_age()

        if year == 2020 or year == 2021 or year == 2022:  # Considering the COVID period
            random_gestational_age += random.randint(2, 4)  # Increase the gestational age by 2 to 4 weeks
            
            if random.random() < 0.80:  # 80% chance of cooling therapy during covid
                # generate_procedure_coolingtherapy(random_datetime)
                df2.loc[len(df2)] = {'year': str(random_datetime)[:4]}

        
        generate_dfs(random_datetime, random_gestational_age)
        generate_observation(random_datetime, random_gestational_age)
        
        if year == 2017 or year == 2018 or year == 2019 or year == 2023:
            if random.random() < 0.60:  # 60% chance of cooling therapy during covid
                # generate_procedure_coolingtherapy(random_datetime)
                df2.loc[len(df2)] = {'year': str(random_datetime)[:4]}


{'resourceType': 'Observation', 'id': '3c3a657f-439e-4db3-97e0-aa8c8f06019c', 'status': 'final', 'category': [{'coding': [{'system': 'gestational-age', 'code': 'gestational-age', 'display': 'Gestational age'}]}], 'code': {'coding': [{'system': 'gestAge', 'code': 'gestAge', 'display': 'Gestational Age'}], 'text': 'Gestational Age'}, 'subject': {'reference': 'https://firelyasserver.azurewebsites.net/Patient/patient-0'}, 'encounter': {'reference': 'no-ref'}, 'effectiveDateTime': '2017-04-15T00:29:31+02:00', 'issued': '2020-08-31T22:13:22.922+02:00', 'valueQuantity': {'value': 38, 'unit': 'weeks'}, 'interpretation': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation', 'code': 'H', 'display': 'High'}]}], 'bodySite': {'coding': [{'system': 'http://snomed.info/sct', 'code': '0', 'display': 'Body part 0'}]}, 'component': [{'code': {'coding': [{'system': 'http://loinc.org', 'code': '0', 'display': 'Component 0'}]}}]}
{'resourceType': 'Observation', 'id':